# Índice
<p>
<div class="lev1 toc-item">
    <a href="#Extracción-de-transacciones-de-2016" data-toc-modified-id="Extracción-de-transacciones-de-2016">
        <span class="toc-item-num">1&nbsp;&nbsp;</span>
        Extracción de transacciones de 2016</a></div>

<div class="lev1 toc-item">
    <a href="#Limpieza-y-transformación-de-transacciones" data-toc-modified-id="Limpieza-y-transformación-de-transacciones">
    <span class="toc-item-num">2&nbsp;&nbsp;</span>
    Limpieza y transformación de transacciones</a></div>
    
<div class="lev1 toc-item">
    <a href="#Extracción-de-datos-de-capacidad-de-plazas" data-toc-modified-id="Extracción-de-datos-de-capacidad-de-plazas">
    <span class="toc-item-num">3&nbsp;&nbsp;</span>
    Extracción de datos de capacidad de plazas</a></div>
    
<div class="lev1 toc-item">
    <a href="#Extracción-de-datos-de-ubicación-geográfica" data-toc-modified-id="Extracción-de-datos-de-ubicación-geográfica">
    <span class="toc-item-num">4&nbsp;&nbsp;</span>
    Extracción de datos de ubicación geográfica</a></div>
    
<div class="lev1 toc-item">
    <a href="#Relación-de-transacciones,-ubicaciones-y-capacidad" data-toc-modified-id="Relación-de-transacciones,-ubicaciones-y-capacidad">
    <span class="toc-item-num">5&nbsp;&nbsp;</span>
    Relación de transacciones, ubicaciones y capacidad</a></div>
          
<div class="lev1 toc-item">
    <a href="#Construcción-de-la-serie-de-ocupación-por-timestamp" data-toc-modified-id="Construcción-de-la-serie-de-ocupación-por-timestamp">
    <span class="toc-item-num">6&nbsp;&nbsp;</span>
    Construcción de la serie de ocupación por timestamp</a></div>

<div class="lev1 toc-item">
    <a href="#Corrección-de-la-capacidad-de-plazas-disponibles" data-toc-modified-id="Corrección-de-la-capacidad-de-plazas-disponibles">
    <span class="toc-item-num">7&nbsp;&nbsp;</span>
    Corrección de la capacidad de plazas disponibles</a></div>

<div class="lev1 toc-item">
    <a href="#Completamos-la-serie-con-las-horas-intermedias-faltantes" data-toc-modified-id="Completamos-la-serie-con-las-horas-intermedias-faltantes">
    <span class="toc-item-num">8&nbsp;&nbsp;</span>
    Completamos la serie con las horas intermedias faltantes</a></div>

<div class="lev1 toc-item">
    <a href="#Completamos-la-serie-con-variables-adicionales" data-toc-modified-id="Completamos-la-serie-con-variables-adicionales">
    <span class="toc-item-num">9&nbsp;&nbsp;</span>
    Completamos la serie con variables adicionales</a></div>

In [1]:
import numpy as np
import pandas as pd
import os
import operator
import seaborn as sns
% matplotlib inline

## Extracción de transacciones de 2016

In [2]:
df = pd.read_csv('./data/ParkingTransaction_2016_cleaned.csv', parse_dates=['TransactionDateTime'])

In [3]:
df['TransactionDateTime'].describe()

count                10935395
unique                5505469
top       2016-04-08 12:03:59
freq                       63
first     2016-01-01 11:49:15
last      2016-12-31 20:00:51
Name: TransactionDateTime, dtype: object

In [4]:
init_transactions = df['TransactionDateTime'].count()

In [5]:
import re

_underscorer1 = re.compile(r'(.)([A-Z][a-z]+)')
_underscorer2 = re.compile('([a-z0-9])([A-Z])')

def camelToSnake(s):
    subbed = _underscorer1.sub(r'\1_\2', s)
    return _underscorer2.sub(r'\1_\2', subbed).lower()

In [6]:
df.columns = df.columns.map(lambda x: camelToSnake(x))

df.sort_values('transaction_date_time', inplace=True)

# creamos nueva columna 'final_date_time'
df['final_date_time'] = df.transaction_date_time.add(pd.to_timedelta(df.duration_mins, unit="m"))

df.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10935395 entries, 8 to 10912795
Data columns (total 12 columns):
unnamed: 0               int64
transaction_id           int64
transaction_date_time    datetime64[ns]
transaction_date         object
time_start               object
time_expired             object
duration_mins            int64
amount                   float64
payment_mean             object
meter_code               int64
element_key              int64
final_date_time          datetime64[ns]
dtypes: datetime64[ns](2), float64(1), int64(5), object(4)
memory usage: 3.4 GB


In [7]:
df = df.drop(columns=['unnamed: 0','transaction_date','amount','payment_mean',
                      'meter_code','time_start','time_expired'])

In [8]:
df.head()

,transaction_id,transaction_date_time,duration_mins,element_key,final_date_time
8,183506786,2016-01-01 11:49:15,0,54730,2016-01-01 11:49:15
4,183511748,2016-01-01 12:36:04,0,88773,2016-01-01 12:36:04
17,183511753,2016-01-01 12:36:08,0,88773,2016-01-01 12:36:08
15,183529739,2016-01-01 12:40:30,0,57354,2016-01-01 12:40:30
12,183527118,2016-01-01 14:52:45,0,76102,2016-01-01 14:52:45


In [9]:
df.tail()

,transaction_id,transaction_date_time,duration_mins,element_key,final_date_time
10909957,313159887,2016-12-31 19:58:49,2,81194,2016-12-31 20:00:49
10918741,313159702,2016-12-31 19:58:53,2,79570,2016-12-31 20:00:53
10910192,313159980,2016-12-31 19:59:24,1,1234,2016-12-31 20:00:24
10907062,313159989,2016-12-31 19:59:45,0,43914,2016-12-31 19:59:45
10912795,313160195,2016-12-31 20:00:51,0,8306,2016-12-31 20:00:51


### Limpieza y transformación de transacciones

In [10]:
df.transaction_id.duplicated().sum()  # no hay transacciones con id duplicado

0

In [11]:
df.isnull().sum()  # no hay valores nulos en el dataset

transaction_id           0
transaction_date_time    0
duration_mins            0
element_key              0
final_date_time          0
dtype: int64

#### Duración incorrecta

In [12]:
# Observamos algunas transacciones con duración = 0
df.duration_mins.loc[df.duration_mins == 0].count()

9086

In [13]:
# Incluso hay algunas transacciones con duración negativa
df.duration_mins.loc[df.duration_mins < 0].count()

1118

In [14]:
# Eliminamos ambos casos que suponen menos de un 0.1% del total
df = df.loc[df.duration_mins > 0]
df = df.drop(columns=['duration_mins'])

#### Transacciones de larga duración

In [15]:
# Procesado de transacciones con distinta fecha de inicio y fin
long_trans = df.loc[df.transaction_date_time.dt.date != df.final_date_time.dt.date]
print(long_trans.shape)
indexes = df.loc[df.transaction_date_time.dt.date != df.final_date_time.dt.date].index.get_values()
df = df.drop(index=indexes, axis=1)

(13403, 4)


In [16]:
long_trans_dup = long_trans.copy()
long_trans_dup = long_trans_dup.loc[long_trans_dup.final_date_time.dt.hour >= 8]
long_trans.final_date_time = long_trans.transaction_date_time.apply(
    lambda x: x.replace(hour=20,minute=0,second=0,microsecond=0))
long_trans_dup.transaction_date_time = long_trans_dup.final_date_time.apply(
    lambda x: x.replace(hour=8,minute=0,second=0,microsecond=0))

In [17]:
long_trans_full = pd.concat([long_trans,long_trans_dup], ignore_index=True, sort=True)
df = pd.concat([df,long_trans_full], ignore_index=True, sort=True)

#### Horario de funcionamiento de los parquímetros

In [18]:
# Eliminamos las transacciones con inicio y fin antes de las 08:00
indexes = df.loc[(df.transaction_date_time.dt.hour < 8) & (df.final_date_time.dt.hour < 8)].index.get_values()
print(indexes.size)
df = df.drop(index=indexes, axis=1)

55413


In [19]:
# Eliminamos las transacciones con inicio y fin después de las 20:00
indexes = df.loc[(df.transaction_date_time.dt.hour >= 20) & (df.final_date_time.dt.hour >= 20)].index.get_values()
print(indexes.size)
df = df.drop(index=indexes, axis=1)

17107


In [20]:
# Redondeamos la hora (sin minutos o segundos) y fijamos a 08:00 la hora mínima de comienzo
df.transaction_date_time = np.where(df.transaction_date_time.dt.hour < 8, 
                                    df.transaction_date_time.apply(lambda x: x.replace(hour=8,minute=0,second=0,microsecond=0)), 
                                    df.transaction_date_time.apply(lambda x: x.replace(minute=0,second=0,microsecond=0)))

In [21]:
# Redondeamos la hora final (sin minutos o segundos) y fijamos a 20:00 la hora máxima final
df.final_date_time = np.where(df.final_date_time.dt.hour > 20,
                              df.final_date_time.apply(lambda x: x.replace(hour=20,minute=0,second=0,microsecond=0)), 
                              df.final_date_time.apply(lambda x: x.replace(minute=0,second=0,microsecond=0)))

In [22]:
# Comprobamos que no hay transacciones fuera del rango horario de uso de los parquímetros (de 8 a 20h)
df.element_key.loc[((df.transaction_date_time.dt.hour > 20) | (df.transaction_date_time.dt.hour < 8)) | 
       ((df.final_date_time.dt.hour > 20) | (df.final_date_time.dt.hour < 8))].count()

0

#### Domingos o festivos

In [23]:
# Observamos algunas transacciones realizadas por error en domingo
df.transaction_date_time.loc[df.transaction_date_time.dt.weekday == 6].count()

239

In [24]:
# Las eliminamos también
df = df.loc[df.transaction_date_time.dt.weekday != 6]

In [25]:
# Y observamos transacciones en días festivos
import holidays

hol = holidays.US(state='WA', years=[2016]).items()
hol_dates = []
for dat, name in sorted(hol):
    hol_dates.append(dat)

df.transaction_date_time.loc[df.transaction_date_time.dt.date.isin(hol_dates)].count()

32871

In [26]:
# Las eliminamos también
df = df.loc[~df.transaction_date_time.dt.date.isin(hol_dates)]

In [27]:
df.shape  

(10822236, 4)

In [28]:
# Con la limpieza hemos reducido el tamaño del dataset inicial en poco más de un 1%
(init_transactions-(df.shape[0]-long_trans_dup.shape[0]))/init_transactions*100

1.0592575759723357

In [29]:
df.to_csv('./data/ParkingTransaction_2016_cleaned_ourETL.csv', index=False)

## Extracción de datos de capacidad de plazas

In [30]:
blocks = pd.read_csv('./data/Blockface_cleaned.csv')

In [31]:
blocks.shape

(13706, 39)

In [32]:
blocks.columns = blocks.columns.map(lambda x: camelToSnake(x))

In [33]:
blocks.loc[blocks.element_key == 1001]

,pay_station_blockface_id,element_key,parking_spaces,paid_parking_area,parking_time_limit_category,peak_hour_start1,peak_hour_end1,peak_hour_start2,peak_hour_end2,paid_area_start_time,...,saturday_start1,saturday_end1,saturday_rate2,saturday_start2,saturday_end2,saturday_rate3,saturday_start3,saturday_end3,start_time_saturday,end_time_saturday
469,7576,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.0,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
2783,10071,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
3017,10262,1001,4.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
4648,11976,1001,4.0,NaN,120.0,06:00:00,09:00:00,15:00:00,18:00:00,NaN,...,00:00:00,00:00:00,0.0,00:00:00,00:00:00,0.0,00:00:00,00:00:00,NaN,NaN
5989,13670,1001,4.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
6301,14348,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,5.0,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
7077,13939,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,4.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00
8059,3405,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10576,1352,1001,7.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11332,6046,1001,5.0,Pioneer Square,120.0,06:00:00,09:00:00,15:00:00,18:00:00,08:00:00,...,08:00:00,11:00:00,3.5,11:00:00,18:00:00,NaN,NaN,NaN,08:00:00,18:00:00


In [34]:
blocks.isnull().sum()

pay_station_blockface_id           0
element_key                        0
parking_spaces                    14
paid_parking_area               1831
parking_time_limit_category     1330
peak_hour_start1               12368
peak_hour_end1                 12368
peak_hour_start2               13209
peak_hour_end2                 13209
paid_area_start_time            1831
paid_area_end_time              1831
effective_start_date               0
effective_end_date              1707
paid_parking_rate               9057
parking_category                   0
load                               0
zone                               0
weekday_rate1                   4677
weekday_start1                  4677
weekday_end1                    4677
weekday_rate2                   4677
weekday_start2                  4677
weekday_end2                    4677
weekday_rate3                   7532
weekday_start3                  7532
weekday_end3                    7532
start_time_weekday              6480
e

Dado que hay algunos casos como el anterior donde hay divergencia en la información de plazas asociadas a un mismo element_key, calculamos la media redondeada a un valor entero de los distintos valores existentes.

In [35]:
park_spaces = blocks[['element_key','parking_spaces','paid_parking_area']]
park_spaces = park_spaces.dropna()
park_spaces = park_spaces.groupby(['element_key','paid_parking_area'])['parking_spaces'].max()

In [36]:
park_spaces = park_spaces.reset_index()

In [37]:
park_spaces.head()

,element_key,paid_parking_area,parking_spaces
0,1001,Pioneer Square,7.0
1,1002,Pioneer Square,9.0
2,1005,Commercial Core,8.0
3,1006,Commercial Core,6.0
4,1009,Commercial Core,5.0


In [38]:
park_spaces[['element_key','paid_parking_area']].duplicated().sum()

0

In [39]:
park_spaces.shape

(1709, 3)

In [40]:
park_spaces[park_spaces.element_key.duplicated(keep=False)]
# Encontramos dos element key con información ambigua del distrito asociado
# Los analizaremos en el siguiente apartado ubicándolos en un mapa

,element_key,paid_parking_area,parking_spaces
194,9762,Commercial Core,4.0
195,9762,Pioneer Square,4.0
1439,78114,Belltown,15.0
1440,78114,Commercial Core,17.0


In [41]:
park_spaces.parking_spaces.describe().astype(str)

count               1709.0
mean     8.537741369221767
std      5.441127087493119
min                    0.0
25%                    5.0
50%                    8.0
75%                   10.0
max                   63.0
Name: parking_spaces, dtype: object

In [42]:
park_spaces.parking_spaces.sort_values(ascending=False).head(10)  
# el valor de 63 podría ser outlier
# comprobamos que no influye en el análisis porque ese EK no está en el fichero de coordenadas

1161    63.0
1685    50.0
754     48.0
1683    47.0
753     45.0
1673    40.0
536     39.0
919     36.0
1682    36.0
1365    34.0
Name: parking_spaces, dtype: float64

In [43]:
park_spaces.element_key.loc[park_spaces.parking_spaces == 0].count()

6

In [44]:
# Eliminamos los registros que no tienen información de la capacidad de plazas
park_spaces = park_spaces.loc[park_spaces.parking_spaces != 0]

In [45]:
park_spaces.head()

,element_key,paid_parking_area,parking_spaces
0,1001,Pioneer Square,7.0
1,1002,Pioneer Square,9.0
2,1005,Commercial Core,8.0
3,1006,Commercial Core,6.0
4,1009,Commercial Core,5.0


## Extracción de datos de ubicación geográfica

In [46]:
coord = pd.read_csv('./data/Coord_EK.csv')

In [47]:
coord.head()

,element_key,latitude,longitude
0,1001,47.602862,-122.334703
1,1002,47.602997,-122.334538
2,1005,47.603602,-122.335382
3,1006,47.603725,-122.335171
4,1009,47.605010,-122.336669


In [48]:
coord.shape

(1517, 3)

In [49]:
coord.duplicated().sum()  # no hay registros duplicados

0

In [50]:
coord.element_key.duplicated().sum()  # tampoco hay duplicados de Element Key

0

In [51]:
coord[['latitude','longitude']].duplicated().sum()  # ni hay ubicaciones repetidas

0

#### Análisis de ubicación de los element_key duplicados en park_spaces

In [52]:
coord.loc[(coord.element_key == 9762) | (coord.element_key == 78114)]

,element_key,latitude,longitude
175,9762,47.602669,-122.336182
1270,78114,47.609074,-122.343332


Utilizamos la web https://www.coordenadas-gps.com para analizar las ubicaciones y decidimos asociar ambos segmentos de calle con el distrito Commercial Core:

In [53]:
park_spaces = park_spaces[(park_spaces.index != 195) & (park_spaces.index != 1439)]  # eliminamos los duplicados

## Relación de transacciones, ubicaciones y capacidad

In [54]:
df.element_key.unique().size

1514

In [55]:
coord.element_key.unique().size

1517

In [56]:
park_spaces.element_key.unique().size

1701

In [57]:
len(set(coord.element_key).intersection(set(df.element_key)))

1445

In [58]:
len(set(park_spaces.element_key).intersection(set(coord.element_key)))

1498

**Mezclamos los datasets:**

In [59]:
# Transacciones con Coordenadas
df_c = pd.merge(df, coord, on='element_key', how='inner', validate='many_to_one')

In [60]:
# Transacciones con Coordenadas y con Capacidad de Plazas disponibles
df_cp = pd.merge(df_c, park_spaces, on='element_key', how='inner', validate='many_to_one')

In [61]:
df.shape, df_c.shape, df_cp.shape

((10822236, 4), (10657805, 6), (10652430, 8))

In [62]:
df_cp.head()

,element_key,final_date_time,transaction_date_time,transaction_id,latitude,longitude,paid_parking_area,parking_spaces
0,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64059668,47.619158,-122.346457,Uptown Triangle,10.0
1,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64059669,47.619158,-122.346457,Uptown Triangle,10.0
2,35693,2016-01-02 14:00:00,2016-01-02 08:00:00,64084672,47.619158,-122.346457,Uptown Triangle,10.0
3,35693,2016-01-02 09:00:00,2016-01-02 08:00:00,183563751,47.619158,-122.346457,Uptown Triangle,10.0
4,35693,2016-01-02 10:00:00,2016-01-02 08:00:00,64084673,47.619158,-122.346457,Uptown Triangle,10.0


In [63]:
df_cp.element_key.unique().size

1443

## Construcción de la serie de ocupación por timestamp

Construimos primero la tabla corta que usaremos para montar las series temporales:

In [64]:
in_cols = ['element_key', 'latitude', 'longitude', 'transaction_date_time', 'parking_spaces','paid_parking_area']
out_cols = ['element_key', 'latitude', 'longitude', 'final_date_time', 'parking_spaces','paid_parking_area']

df_in = df_cp[in_cols].reset_index(drop=True)
df_in['timestamp_sign'] = 1.0
df_in.rename(columns={'transaction_date_time': 'timestamp'}, inplace=True)

df_out = df_cp[out_cols].reset_index(drop=True)
df_out['timestamp_sign'] = - 1.0
df_out.rename(columns={'final_date_time': 'timestamp'}, inplace=True)  # inplace = True, value of copy is ignored

In [65]:
df_in.head()

,element_key,latitude,longitude,timestamp,parking_spaces,paid_parking_area,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
1,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
2,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
3,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
4,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0


In [66]:
df_out.head()

,element_key,latitude,longitude,timestamp,parking_spaces,paid_parking_area,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 10:00:00,10.0,Uptown Triangle,-1.0
1,35693,47.619158,-122.346457,2016-01-02 10:00:00,10.0,Uptown Triangle,-1.0
2,35693,47.619158,-122.346457,2016-01-02 14:00:00,10.0,Uptown Triangle,-1.0
3,35693,47.619158,-122.346457,2016-01-02 09:00:00,10.0,Uptown Triangle,-1.0
4,35693,47.619158,-122.346457,2016-01-02 10:00:00,10.0,Uptown Triangle,-1.0


In [67]:
full_transactions = pd.concat([df_in, df_out])
full_transactions = full_transactions.sort_values('timestamp', ascending=True)
full_transactions = full_transactions.reset_index(drop=True)

In [68]:
full_transactions.head()

,element_key,latitude,longitude,timestamp,parking_spaces,paid_parking_area,timestamp_sign
0,35693,47.619158,-122.346457,2016-01-02 08:00:00,10.0,Uptown Triangle,1.0
1,46254,47.611498,-122.343153,2016-01-02 08:00:00,10.0,Belltown,1.0
2,46254,47.611498,-122.343153,2016-01-02 08:00:00,10.0,Belltown,1.0
3,46254,47.611498,-122.343153,2016-01-02 08:00:00,10.0,Belltown,1.0
4,25710,47.613757,-122.344972,2016-01-02 08:00:00,11.0,Belltown,1.0


In [69]:
df_in.shape, df_out.shape, full_transactions.shape

((10652430, 7), (10652430, 7), (21304860, 7))

#### Añadimos porcentaje de ocupación

In [70]:
# COMANDO A REVISAR OPTIMIZACIÓN - LE HA COSTADO
full_transactions['occupation'] = full_transactions.groupby(['element_key', 'timestamp']).timestamp_sign.transform(
    lambda series: series.cumsum())

In [71]:
full_transactions = full_transactions.reset_index(drop=True)

In [72]:
full_transactions.drop_duplicates(subset=['element_key','timestamp'], keep='last', inplace=True)

In [73]:
full_transactions['day_year'] = full_transactions.timestamp.dt.dayofyear

In [74]:
full_transactions['occu_cum'] = full_transactions.groupby(['element_key','day_year']).occupation.agg('cumsum')

In [75]:
full_transactions['occupation_perc'] = (full_transactions.occu_cum / full_transactions.parking_spaces * 100.0).map(
    "{0:.2f}".format).astype(float)

In [76]:
full_transactions.element_key.loc[full_transactions.occupation_perc > 100].count()

204324

In [77]:
full_transactions.element_key.loc[full_transactions.occupation_perc > 100].unique().size

1174

In [78]:
full_transactions.shape

(4124933, 11)

## Corrección de la capacidad de plazas disponibles

In [79]:
new_park_spaces = pd.read_csv('./data/StreetParking.csv')

In [80]:
new_park_spaces.head()

,element_key,parking_category,total_nopark,total_zones,parking_spaces,total_spaces
0,32018,Paid Parking,8,1,6,15
1,1042,No Parking Allowed,5,1,0,6
2,48129,Paid Parking,6,1,4,11
3,47962,Paid Parking,8,2,7,17
4,47966,Paid Parking,5,1,4,10


In [81]:
# Eliminamos los registros que no tienen información de la capacidad de plazas
new_park_spaces = new_park_spaces.loc[new_park_spaces.total_spaces != 0]

In [82]:
new_park_spaces = pd.merge(new_park_spaces, park_spaces, on='element_key', how='inner', validate='many_to_one')

In [83]:
new_park_spaces.head()

,element_key,parking_category,total_nopark,total_zones,parking_spaces_x,total_spaces,paid_parking_area,parking_spaces_y
0,32018,Paid Parking,8,1,6,15,Belltown,6.0
1,48129,Paid Parking,6,1,4,11,Pioneer Square,6.0
2,47962,Paid Parking,8,2,7,17,Commercial Core,8.0
3,47966,Paid Parking,5,1,4,10,Commercial Core,9.0
4,8617,Paid Parking,7,2,4,13,Denny Triangle,4.0


In [84]:
new_park_spaces.element_key.loc[new_park_spaces.parking_spaces_x == new_park_spaces.parking_spaces_y].count()

1055

In [85]:
new_park_spaces.shape

(1694, 8)

In [86]:
new_park_spaces = new_park_spaces.drop(columns=['total_nopark','total_zones','parking_spaces_x','parking_spaces_y'])

In [87]:
new_park_spaces.total_spaces.value_counts()

16    151
15    138
14    133
17    125
12    114
13    114
11    107
18     87
10     85
19     62
8      56
9      53
21     40
22     39
23     39
7      37
6      36
20     35
4      31
5      24
24     22
3      21
25     19
2      18
26     14
1      12
28     10
33     10
27      9
29      7
31      7
35      7
32      4
34      3
38      3
30      3
41      3
40      2
37      2
36      2
44      2
50      2
51      1
53      1
43      1
42      1
39      1
54      1
Name: total_spaces, dtype: int64

In [88]:
# Eliminamos aquellos parquímetros con un número de plazas inferior a 4
new_park_spaces = new_park_spaces.loc[new_park_spaces.total_spaces > 4]

In [89]:
df_cnp = pd.merge(df_c, new_park_spaces, on='element_key', how='inner', validate='many_to_one')

In [90]:
df_cnp.shape

(10625789, 9)

In [91]:
df_cnp.element_key.unique().size  # se corrige la capacidad de plazas para 1436-1055 = 381 element keys

1436

In [92]:
new_park_spaces = new_park_spaces.drop(columns=['paid_parking_area'])
full_transactions = full_transactions.drop(columns=['occupation_perc','timestamp_sign','occupation','parking_spaces'])
full_transactions = pd.merge(full_transactions, new_park_spaces, on='element_key', how='inner', validate='many_to_one')
full_transactions['occupation_perc'] = (full_transactions.occu_cum / full_transactions.total_spaces * 100.0).map(
    "{0:.2f}".format).astype(float)

In [93]:
full_transactions.element_key.loc[full_transactions.occupation_perc > 100].count()

9808

In [94]:
full_transactions.element_key.loc[full_transactions.occupation_perc > 100].unique().size

187

In [95]:
# Eliminamos los registros asociados a element_keys con ocupación > 100%
ek_del = full_transactions.element_key.loc[full_transactions.occupation_perc > 100].unique()
full_transactions = full_transactions.loc[~full_transactions.element_key.isin(ek_del)]

In [96]:
full_transactions.element_key.unique().size

1249

In [97]:
full_transactions.parking_category.value_counts()/full_transactions.shape[0]

Paid Parking               0.933928
Restricted Parking Zone    0.046733
Carpool Parking            0.011140
No Parking Allowed         0.008199
Name: parking_category, dtype: float64

In [98]:
# Nos quedamos con la categoría mayoritaria Paid Parking
full_transactions = full_transactions.loc[full_transactions.parking_category == 'Paid Parking']

In [99]:
# Comprobamos que la ocupación es 0 para las transacciones con hora = 20
full_transactions.occupation_perc.loc[full_transactions.timestamp.dt.hour == 20].unique()

array([0.])

In [100]:
# Y las eliminamos
full_transactions = full_transactions.loc[full_transactions.timestamp.dt.hour < 20]

In [101]:
full_transactions = full_transactions.drop(columns=['occu_cum','parking_category','total_spaces'])

In [102]:
full_transactions.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc
1385,75865,47.598052,-122.326466,2016-01-02 08:00:00,Chinatown/ID,2,11.76
1386,75865,47.598052,-122.326466,2016-01-02 09:00:00,Chinatown/ID,2,11.76
1387,75865,47.598052,-122.326466,2016-01-02 10:00:00,Chinatown/ID,2,11.76
1388,75865,47.598052,-122.326466,2016-01-02 11:00:00,Chinatown/ID,2,29.41
1389,75865,47.598052,-122.326466,2016-01-02 12:00:00,Chinatown/ID,2,52.94


In [109]:
full_transactions.shape

(3133583, 7)

In [211]:
df_cnp = df_cnp.loc[df_cnp.parking_category == 'Paid Parking']
df_cnp = df_cnp.drop(columns=['transaction_id','parking_category'])
df_cnp = df_cnp.loc[~df_cnp.element_key.isin(ek_del)]
df_cnp = df_cnp.reset_index(drop=True)

In [101]:
df_cnp.head()

,element_key,final_date_time,transaction_date_time,latitude,longitude,total_spaces,paid_parking_area
0,53549,2016-01-02 10:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union
1,53549,2016-01-02 17:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union
2,53549,2016-01-02 09:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union
3,53549,2016-01-02 09:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union
4,53549,2016-01-02 09:00:00,2016-01-02 08:00:00,47.628175,-122.341132,50,South Lake Union


In [102]:
# Guardamos las transacciones asociadas con ubicaciones y capacidad de plazas
df_cnp.to_csv('./data/ParkingTrans2016_with_Coord_Spaces', index=False)

## Completamos la serie con las horas intermedias faltantes

In [107]:
all_eks = sorted(full_transactions.element_key.unique())
days = sorted(full_transactions.day_year.unique())
hours = list(range(8,20))

full_transactions_ext = full_transactions.copy()

for ek in all_eks:
    ek_days = full_transactions.loc[full_transactions.element_key == ek].day_year.unique()
    ek_days_faltan = list(set(days) ^ set(ek_days))
    for d in ek_days:
        if d not in ek_days_faltan:
            ek_d_hours = list(sorted(full_transactions.loc[(full_transactions.element_key == ek) &
                                               (full_transactions.day_year == d)].timestamp.dt.hour.unique()))
            if len(ek_d_hours) != 12:
                ek_hours_faltan = list(set(hours) ^ set(ek_d_hours))
                if (ek_d_hours[0] != 8):
                    for h in range(8,ek_d_hours[0]):
                        ek_hours_faltan.remove(h)
                        new_row = full_transactions.loc[(full_transactions.element_key == ek) &
                                                    (full_transactions.day_year == d) &
                                                    (full_transactions.timestamp.dt.hour == ek_d_hours[0])]
                        i = new_row.index[0]
                        new_row.at[i,'timestamp'] = new_row.at[i,'timestamp'].replace(hour=h)
                        new_row.at[i,'occupation_perc'] = 0
                        full_transactions_ext = full_transactions_ext.append(new_row, ignore_index = True)
                if (ek_d_hours[-1] != 19):
                    for h in range(ek_d_hours[-1]+1,20):
                        ek_hours_faltan.remove(h)
                        new_row = full_transactions.loc[(full_transactions.element_key == ek) &
                                                    (full_transactions.day_year == d) &
                                                    (full_transactions.timestamp.dt.hour == ek_d_hours[0])]
                        i = new_row.index[0]
                        new_row.at[i,'timestamp'] = new_row.at[i,'timestamp'].replace(hour=h)
                        new_row.at[i,'occupation_perc'] = 0
                        full_transactions_ext = full_transactions_ext.append(new_row, ignore_index = True)
                for h in ek_hours_faltan:
                    new_h = max(list(filter(lambda x: x - h < 0, ek_d_hours)))
                    new_row = full_transactions.loc[(full_transactions.element_key == ek) &
                                                    (full_transactions.day_year == d) &
                                                    (full_transactions.timestamp.dt.hour == new_h)]
                    i = new_row.index[0]
                    new_row.at[i,'timestamp'] = new_row.at[i,'timestamp'].replace(hour=h)
                    full_transactions_ext = full_transactions_ext.append(new_row, ignore_index = True)
            
full_transactions_ext = full_transactions_ext.sort_values(['element_key','timestamp'])            

In [115]:
full_transactions_ext.shape[0] - full_transactions.shape[0]  # registros añadidos

718081

In [108]:
full_transactions_ext.to_csv('./data/FullTrans2016_extended_global.csv', index=False)

## Completamos la serie con variables adicionales

#### Datos meteorológicos globales

In [265]:
meteo = pd.read_csv('./data/seattleWeather_1948-2017.csv', parse_dates=['DATE'])
meteo.columns = meteo.columns.map(lambda x: camelToSnake(x))

meteo = meteo.loc[meteo['date'].dt.year == 2016]
meteo['day_year'] = meteo.date.dt.dayofyear

meteo.tmax = ((meteo.tmax - 32) * 5.0/9.0).map("{0:.2f}".format).astype(float)
meteo.tmin = ((meteo.tmin - 32) * 5.0/9.0).map("{0:.2f}".format).astype(float)

In [266]:
meteo.head()

,date,prcp,tmax,tmin,rain,day_year
24837,2016-01-01,0.00,7.78,-2.22,False,1
24838,2016-01-02,0.00,5.56,-3.89,False,2
24839,2016-01-03,0.02,4.44,-0.56,True,3
24840,2016-01-04,0.15,3.33,1.67,True,4
24841,2016-01-05,0.11,7.78,2.22,True,5


In [267]:
meteo.shape

(366, 6)

In [268]:
ft_ext_meteo = pd.merge(full_transactions_ext, meteo, on='day_year', how='left', validate='many_to_one')

In [269]:
ft_ext_meteo.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc,date,prcp,tmax,tmin,rain
0,1001,47.602862,-122.334703,2016-01-02 08:00:00,Pioneer Square,2,0.0,2016-01-02,0.0,5.56,-3.89,False
1,1001,47.602862,-122.334703,2016-01-02 09:00:00,Pioneer Square,2,0.0,2016-01-02,0.0,5.56,-3.89,False
2,1001,47.602862,-122.334703,2016-01-02 10:00:00,Pioneer Square,2,0.0,2016-01-02,0.0,5.56,-3.89,False
3,1001,47.602862,-122.334703,2016-01-02 11:00:00,Pioneer Square,2,0.0,2016-01-02,0.0,5.56,-3.89,False
4,1001,47.602862,-122.334703,2016-01-02 12:00:00,Pioneer Square,2,0.0,2016-01-02,0.0,5.56,-3.89,False


In [272]:
ft_ext_meteo = ft_ext_meteo.drop(columns=['date','rain'])

#### Sensores de temperatura

In [243]:
weather_stations = pd.read_csv('./data/Coord_EK_stations.csv')
weather_stations.head()
weather_stations = weather_stations.drop(columns=['latitude', 'longitude'])

In [282]:
ft_ext_meteo_ws = pd.merge(ft_ext_meteo, weather_stations, on='element_key', how='left', validate='many_to_one')

In [283]:
ft_ext_meteo_ws.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc,prcp,tmax,tmin,station_closest
0,1001,47.602862,-122.334703,2016-01-02 08:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5
1,1001,47.602862,-122.334703,2016-01-02 09:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5
2,1001,47.602862,-122.334703,2016-01-02 10:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5
3,1001,47.602862,-122.334703,2016-01-02 11:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5
4,1001,47.602862,-122.334703,2016-01-02 12:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5


In [285]:
ft_ext_meteo_ws.timestamp = pd.to_datetime(ft_ext_meteo_ws.timestamp)

In [298]:
rwis = pd.read_csv('./data/RWIS_completed.csv', parse_dates=['timestamp'])

In [299]:
rwis.head()

,station_closest,timestamp,air_temp,road_temp
0,5,2016-01-01 08:00:00,2.03,-2.63
1,5,2016-01-01 09:00:00,1.99,-2.27
2,5,2016-01-01 10:00:00,2.08,-0.89
3,5,2016-01-01 11:00:00,2.28,2.44
4,5,2016-01-01 12:00:00,2.57,4.87


In [300]:
ft_ext_meteo_ws_rwis = pd.merge(ft_ext_meteo_ws, rwis, on=['timestamp','station_closest'], how='left', 
                                validate='many_to_one')

In [301]:
ft_ext_meteo_ws_rwis.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc,prcp,tmax,tmin,station_closest,air_temp,road_temp
0,1001,47.602862,-122.334703,2016-01-02 08:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,1.65,-2.90
1,1001,47.602862,-122.334703,2016-01-02 09:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,1.58,-2.63
2,1001,47.602862,-122.334703,2016-01-02 10:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,1.63,-1.10
3,1001,47.602862,-122.334703,2016-01-02 11:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,1.89,2.37
4,1001,47.602862,-122.334703,2016-01-02 12:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,2.10,4.12


#### Lugares deportivos y de interés cultural

In [303]:
cult_sport = pd.read_csv('./data/Coord_cult_&_sport.csv')

In [304]:
cult_sport.head()

,element_key,latitude,longitude,poi,baseball,tennis,basket,soccer
0,1001,47.602862,-122.334703,1,0,0,0,0
1,1002,47.602997,-122.334538,1,0,0,0,0
2,1005,47.603602,-122.335382,0,0,0,0,0
3,1006,47.603725,-122.335171,0,0,0,0,0
4,1009,47.605010,-122.336669,1,0,0,0,0


In [308]:
cult_sport = cult_sport.drop(columns=['latitude', 'longitude'])
ft_ext_met_ws_rwis_cs = pd.merge(ft_ext_meteo_ws_rwis, cult_sport, on=['element_key'], 
                             how='left', validate='many_to_one')

In [309]:
ft_ext_met_ws_rwis_cs.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc,prcp,tmax,tmin,station_closest,air_temp,road_temp,poi,baseball,tennis,basket,soccer
0,1001,47.602862,-122.334703,2016-01-02 08:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,1.65,-2.90,1,0,0,0,0
1,1001,47.602862,-122.334703,2016-01-02 09:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,1.58,-2.63,1,0,0,0,0
2,1001,47.602862,-122.334703,2016-01-02 10:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,1.63,-1.10,1,0,0,0,0
3,1001,47.602862,-122.334703,2016-01-02 11:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,1.89,2.37,1,0,0,0,0
4,1001,47.602862,-122.334703,2016-01-02 12:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,5,2.10,4.12,1,0,0,0,0


#### Eventos

In [129]:
events = pd.read_csv('./data/Events_2016.csv')

In [310]:
events.head()

,Latitude,Longitude,day_year
0,47.611543,-122.33263,105
1,47.611543,-122.33263,106
2,47.601130,-122.32980,137
3,47.628560,-122.33979,137
4,47.636290,-122.35922,137


In [131]:
# Función para calcular la distancia entre dos puntos teniendo en cuenta la curvatura de la tierra (no Euclidea)
from math import radians, cos, sin, asin, sqrt, acos

def haversine(lon1, lat1, lon2, lat2):
    # convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])
    # haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6373 # Radius of earth kilometers
    return c * r

In [311]:
# Creamos una nueva columna que indica si hay un evento próximo al element_key
# Una vez que un element_key tiene un punto cercano (<=75 metros) no es necesario revisar otros (break)
ft_ext_met_ws_rwis_cs['event'] = 0
for ft_index, ft_row in ft_ext_met_ws_rwis_cs.iterrows():
    events_sub = events.loc[events.day_year == ft_row.day_year]
    for e_index, e_row in events_sub.iterrows():
        dist = haversine(e_row.Longitude, e_row.Latitude, ft_row.longitude, ft_row.latitude)
        if dist <= 0.075:
            full_trans_ext_met_cs.at[ft_index,'event'] = 1
            break

#### Polución

In [155]:
aqds = pd.read_csv('./data/Air_Quality_Data_Seattle_2016.csv')

In [313]:
aqds.head()

,day_year,no2,co,pm2_5,o3
0,1,75.262667,858.75,26.685000,61.0
1,2,71.001333,1030.50,19.875000,83.0
2,3,72.568000,801.50,14.281250,62.0
3,4,64.860000,629.75,11.047368,49.0
4,5,70.938667,1030.50,14.912500,45.0


In [314]:
ft_ext_met_ws_rwis_cs_aq = pd.merge(ft_ext_met_ws_rwis_cs, aqds, on=['day_year'], how='left', validate='many_to_one')

In [316]:
ft_ext_met_ws_rwis_cs_aq.head()

,element_key,latitude,longitude,timestamp,paid_parking_area,day_year,occupation_perc,prcp,tmax,tmin,...,poi,baseball,tennis,basket,soccer,event,no2,co,pm2_5,o3
0,1001,47.602862,-122.334703,2016-01-02 08:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
1,1001,47.602862,-122.334703,2016-01-02 09:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
2,1001,47.602862,-122.334703,2016-01-02 10:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
3,1001,47.602862,-122.334703,2016-01-02 11:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0
4,1001,47.602862,-122.334703,2016-01-02 12:00:00,Pioneer Square,2,0.0,0.0,5.56,-3.89,...,1,0,0,0,0,0,71.001333,1030.5,19.875,83.0


In [315]:
ft_ext_met_ws_rwis_cs_aq.columns

Index(['element_key', 'latitude', 'longitude', 'timestamp',
       'paid_parking_area', 'day_year', 'occupation_perc', 'prcp', 'tmax',
       'tmin', 'station_closest', 'air_temp', 'road_temp', 'poi', 'baseball',
       'tennis', 'basket', 'soccer', 'event', 'no2', 'co', 'pm2_5', 'o3'],
      dtype='object')

In [317]:
ft_ext_met_ws_rwis_cs_aq.isnull().sum()

element_key          0
latitude             0
longitude            0
timestamp            0
paid_parking_area    0
day_year             0
occupation_perc      0
prcp                 0
tmax                 0
tmin                 0
station_closest      0
air_temp             0
road_temp            0
poi                  0
baseball             0
tennis               0
basket               0
soccer               0
event                0
no2                  0
co                   0
pm2_5                0
o3                   0
dtype: int64

In [319]:
ft_ext_met_ws_rwis_cs_aq = ft_ext_met_ws_rwis_cs_aq.drop(columns=['station_closest'])
ft_ext_met_ws_rwis_cs_aq.to_csv('./data/Serie_Total2016_ext.csv', index=False)

In [320]:
ft_ext_met_ws_rwis_cs_aq.shape, ft_ext_met_ws_rwis_cs.shape, ft_ext_meteo_ws_rwis.shape, ft_ext_meteo_ws.shape, ft_ext_meteo.shape, full_transactions_ext.shape

((3851664, 22),
 (3851664, 19),
 (3851664, 13),
 (3851664, 11),
 (3851664, 10),
 (3851664, 7))

## Filtramos la serie

#### Filtrado de Element Keys con todos los días hábiles

In [322]:
days = sorted(ft_ext_met_ws_rwis_cs_aq.day_year.unique())

dias_faltan = []
ek_all_days = []

for ek in all_eks:
    ek_days = ft_ext_met_ws_rwis_cs_aq.loc[ft_ext_met_ws_rwis_cs_aq.element_key == ek].day_year.unique()
    ek_days_faltan = list(set(days) ^ set(ek_days))
    if len(ek_days_faltan) != 0:
        dias_faltan.append((ek, ek_days_faltan))
    else:
        ek_all_days.append(ek)

In [323]:
len(ek_all_days)

217

In [324]:
full_trans_ext_alldays = ft_ext_met_ws_rwis_cs_aq.loc[
    ft_ext_met_ws_rwis_cs_aq.element_key.isin(ek_all_days)]

In [332]:
full_trans_ext_alldays.shape

(791616, 22)

In [325]:
full_trans_ext_alldays.to_csv('./data/SerieTotal2016_ext_alldays.csv', index=False)

#### Filtrado Top 100 de Element Keys con más transacciones

In [326]:
top100_list = df_cnp.groupby('element_key').total_spaces.count().sort_values(
    ascending=False).head(100).index.get_values()

In [327]:
top100_list

array([ 1234, 35845, 35850, 13213, 54737, 35846, 31850, 31849, 63125,
        9474, 35682, 81210, 32266, 24413, 56654, 30698, 54733, 76961,
       18622, 69446, 69489,  9141, 35681, 85385, 46410, 54730, 24041,
       24270, 87257, 13549, 46254, 13793,  9510,  1021,  1046, 41030,
       46409, 69445, 24405, 13214, 49654, 58514, 34214, 12289, 31493,
       70866, 63126, 16942, 76962, 37134, 81209, 74025, 85381, 28949,
       79741, 14677, 35849, 81297, 69098, 24557, 88773,  1018, 75865,
       74029, 32265,  9461, 58513, 37177, 59958, 51485,  9473,  1433,
       75169, 12765, 88634, 81117, 24038,  2926, 34938, 25714,  7261,
       11118, 86449,  1045, 11878, 52525,  8001, 70746, 81421, 24037,
       81194, 62458, 24042, 24265,  1037, 94602, 65817, 59962, 25710,
       24266])

In [328]:
full_trans_ext_top100 = ft_ext_met_ws_rwis_cs_aq.loc[
    ft_ext_met_ws_rwis_cs_aq.element_key.isin(top100_list)]

In [335]:
full_trans_ext_top100.shape

(363744, 22)

In [329]:
full_trans_ext_top100.to_csv('./data/SerieTotal2016_ext_top100.csv', index=False)

In [330]:
len(set(top100_list) & set(ek_all_days))

36

In [331]:
for (ek, ek_list) in dias_faltan:
    if ek in top100_list:
        print (ek, ek_list)

1018 [114, 142, 177, 247]
1021 [247]
1234 [204]
2926 [247]
7261 [247]
8001 [176]
9141 [107]
9461 [247]
9473 [177, 247]
9474 [177, 247]
11118 [247]
12765 [247]
13213 [142]
13214 [142]
16942 [142]
24037 [247]
24038 [247]
24041 [247]
24042 [247]
24405 [177, 205]
24413 [247]
25710 [247]
25714 [247, 349]
28949 [247]
30698 [247]
31493 [247]
31849 [191]
31850 [191]
32265 [177, 247]
32266 [177, 247]
34214 [247, 317]
35845 [142]
35846 [142]
35849 [142]
35850 [142]
46254 [247]
46409 [247]
46410 [247]
49654 [247]
51485 [212, 247]
52525 [247]
54730 [247]
54733 [247, 310]
54737 [177, 247]
56654 [191]
58513 [142]
58514 [142]
59962 [177, 204, 205]
63126 [359]
65817 [247]
69098 [247]
70746 [247, 282]
70866 [121, 247]
74025 [247]
74029 [247]
75169 [247]
75865 [198, 247]
81194 [114, 247]
81210 [142]
81297 [247]
81421 [147, 212, 246, 247]
87257 [191]
88634 [198, 247, 254]
88773 [247]
